In [953]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OrdinalEncoder, LabelEncoder, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.feature_extraction.text import TfidfVectorizer

In [954]:
trainee_path = "C:\\Users\\gabme\\Downloads\\IEEE\\Gestão de Pessoas\\Trainee 2024.1\\Formulário Trainee 2024.1.csv\\Formulário Trainee 2024.1.csv"
member_path = "C:\\Users\\gabme\\Downloads\\IEEE\\Gestão de Pessoas\\Trainee 2024.1\\Formulário dos padrinhos.csv\\Formulário dos padrinhos.csv"

trainee_df = pd.read_csv(trainee_path)
member_df = pd.read_csv(member_path)

trainee_df.describe()

,Carimbo de data/hora,Qual o seu nome e sobrenome?,Insira seu gênero,Qual o seu curso?,Qual o seu semestre?,Qual sua forma preferida de comunicação?,Qual sua principal motivação ao entrar no ramo estudantil?,O que você prefere fazer no seu tempo livre?,Você se considera uma pessoa tímida?,Quais são suas áreas de interesse?,Descreva em um parágrafo a sua personalidade
count,24,24,24,24,24,24,24,24,24,24,24
unique,24,24,2,7,4,2,5,7,3,23,24
top,2024/04/06 3:02:37 PM GMT-3,Matheus Parente Reis,Feminino,Engenharia Elétrica,1°,Reuniões presenciais,Desenvolver hard skills participando de projet...,"Ler algum livro, assistir uma série ou filme e...","Eu não me considero tímido, mas sou extrovertido",Inteligência Artificial;Automação e Robótica;C...,"Comunicativo, amigável, curioso e ansioso"
freq,1,1,13,9,14,19,13,8,10,2,1


In [955]:
names = ['Miriam Uchoa', 'Sanderly Teles', 'Laura Beatriz Gomes Ferreira', 'Luana Sousa Andrade Farias', 'Antônia Marília de Oliveira Coelho']

for name in names:
    trainee_df = trainee_df[trainee_df['Qual o seu nome e sobrenome?'] != name]

In [956]:
member_df = member_df.drop('Carimbo de data/hora', axis=1)
trainee_df = trainee_df.drop('Carimbo de data/hora', axis=1)
trainee_df.columns

Index(['Qual o seu nome e sobrenome?', 'Insira seu gênero',
       'Qual o seu curso?', 'Qual o seu semestre?',
       'Qual sua forma preferida de comunicação?',
       'Qual sua principal motivação ao entrar no ramo estudantil?',
       'O que você prefere fazer no seu tempo livre?',
       'Você se considera uma pessoa tímida?',
       'Quais são suas áreas de interesse?',
       'Descreva em um parágrafo a sua personalidade'],
      dtype='object')

In [957]:
member_personality = pd.DataFrame({'Textos' : member_df['Descreva em um parágrafo a sua personalidade']})
member_df = member_df.drop('Descreva em um parágrafo a sua personalidade', axis=1)

trainee_personality = pd.DataFrame({'Textos' : trainee_df['Descreva em um parágrafo a sua personalidade']})
trainee_df = trainee_df.drop('Descreva em um parágrafo a sua personalidade', axis=1)

member_interests = pd.DataFrame({'Textos' : member_df['Quais são suas áreas de interesse?']})
member_df = member_df.drop('Quais são suas áreas de interesse?', axis=1)

trainee_interests = pd.DataFrame({'Textos' : trainee_df['Quais são suas áreas de interesse?']})
trainee_df = trainee_df.drop('Quais são suas áreas de interesse?', axis=1)


trainee_names = trainee_df['Qual o seu nome e sobrenome?'].values.tolist()
member_names = member_df['Qual o seu nome e sobrenome?'].values.tolist()

trainee_df = trainee_df.drop('Qual o seu nome e sobrenome?', axis = 1)
member_df = member_df.drop('Qual o seu nome e sobrenome?', axis = 1)

In [958]:
unknow_categories_mapping = {
    'Participar de atividades ao ar livre como ir à praia ou a um parque' : 'Sair com amigos para algum lugar agitado'
} 

In [959]:
mask = member_df['O que você prefere fazer no seu tempo livre?'].isin(unknow_categories_mapping.keys())
member_df.loc[mask, 'O que você prefere fazer no seu tempo livre?'] = member_df.loc[mask, 'O que você prefere fazer no seu tempo livre?'].map(unknow_categories_mapping)

In [960]:
text_transformer = TfidfVectorizer()

t_personality_vector = text_transformer.fit_transform(trainee_personality['Textos'])
m_personality_vector = text_transformer.transform(member_personality['Textos'])

t_interests_vector = text_transformer.fit_transform(trainee_interests['Textos'])
m_interests_vector = text_transformer.transform(member_interests['Textos'])

print(t_personality_vector.shape)

(21, 528)


In [961]:
#Transformando

ord_columns = ['Qual o seu curso?', 'Qual sua principal motivação ao entrar no ramo estudantil?', 'O que você prefere fazer no seu tempo livre?', 'Você se considera uma pessoa tímida?', 'Qual o seu semestre?']
hot_columns = ['Insira seu gênero', 'Qual sua forma preferida de comunicação?']

pipeline = ColumnTransformer([
    ('ord_encoder', OrdinalEncoder(), ord_columns),
    ('hot_encoder', OneHotEncoder(drop='first'), hot_columns)
    ])

trainee_encoded = pipeline.fit_transform(trainee_df)
member_encoded = pipeline.transform(member_df)

encoded_columns = pipeline.get_feature_names_out()

t_encoded_df = pd.DataFrame(trainee_encoded, columns=encoded_columns)
m_encoded_df = pd.DataFrame(member_encoded, columns=encoded_columns)

t_encoded_df.head()

,ord_encoder__Qual o seu curso?,ord_encoder__Qual sua principal motivação ao entrar no ramo estudantil?,ord_encoder__O que você prefere fazer no seu tempo livre?,ord_encoder__Você se considera uma pessoa tímida?,ord_encoder__Qual o seu semestre?,hot_encoder__Insira seu gênero_Masculino,hot_encoder__Qual sua forma preferida de comunicação?_Reuniões presenciais
0,4.0,3.0,5.0,1.0,0.0,1.0,1.0
1,5.0,4.0,4.0,1.0,1.0,1.0,1.0
2,3.0,1.0,3.0,2.0,3.0,0.0,1.0
3,1.0,3.0,0.0,2.0,1.0,0.0,1.0
4,4.0,3.0,3.0,2.0,0.0,1.0,1.0


In [962]:
def same_gender(member_df, trainee_df, t_index, m_index):
    return member_df['hot_encoder__Insira seu gênero_Masculino'].iloc[m_index] == trainee_df['hot_encoder__Insira seu gênero_Masculino'].iloc[t_index]

In [963]:
#Calculadora de distâncias
def euclidian_distance(member_df, trainee_df, t_index, m_index):
    trainee_info = trainee_df.iloc[t_index].values
    member_info = member_df.iloc[m_index].values
    
    distance = trainee_info - member_info
    euclides = np.linalg.norm(distance)
    
    return euclides


def cos_distance(m, t, m_index, t_index):
    
    t_arr = t[t_index].toarray().squeeze()
    m_arr = m[m_index].toarray().squeeze()
    
    product = np.dot(t_arr, m_arr)
    
    norm_t = np.linalg.norm(t_arr)
    norm_m = np.linalg.norm(m_arr)
    
    similarity = product/(norm_t * norm_m)
    
    return similarity * 100

In [964]:
similarity = []

for t_range in range(len(trainee_names)):
    t_index = int(t_range)
    for m_range in range(len(member_names)):
        m_index = int(m_range)
        if same_gender(m_encoded_df, t_encoded_df, t_index, m_index):
            distance = euclidian_distance(m_encoded_df, t_encoded_df, t_index, m_index)
            per_cos = cos_distance(m_personality_vector, t_personality_vector, m_index, t_index)
            int_cos = cos_distance(m_interests_vector, t_interests_vector, m_index, t_index)
            
            similarity.append((trainee_names[t_index], member_names[m_index], distance, per_cos, int_cos))      

In [965]:
def find_index(df, name):
    return df.index[df['Qual o seu nome e sobrenome?'] == name].tolist()

In [966]:
for same in similarity:
     t_name, m_name, distance, per_cos, int_cos = same
     print(f"Trainee: {t_name}, Membro: {m_name}, Distância: {distance}, Cosseno da Personalidade: {per_cos}, Cosseno do interesse: {int_cos}")

Trainee: Matheus Parente Reis , Membro: Júlio Schneiders, Distância: 6.0, Cosseno da Personalidade: 0.0, Cosseno do interesse: 73.33695331520356
Trainee: Matheus Parente Reis , Membro: Gabriel Amorim, Distância: 1.7320508075688772, Cosseno da Personalidade: 0.0, Cosseno do interesse: 94.62534197697742
Trainee: Matheus Parente Reis , Membro: Vinicius Lavor, Distância: 3.1622776601683795, Cosseno da Personalidade: 0.0, Cosseno do interesse: 88.39308472768197
Trainee: Matheus Parente Reis , Membro: Diego Lima dos Santos , Distância: 5.830951894845301, Cosseno da Personalidade: 0.0, Cosseno do interesse: 21.545703675946818
Trainee: Matheus Parente Reis , Membro: Pedro Rocha, Distância: 3.3166247903554, Cosseno da Personalidade: 0.0, Cosseno do interesse: 94.62534197697742
Trainee: Matheus Parente Reis , Membro: Benno Brito, Distância: 2.8284271247461903, Cosseno da Personalidade: 0.0, Cosseno do interesse: 33.17358937136151
Trainee: Matheus Parente Reis , Membro: Emirton Queiroga, Distânci